#### Part 1:
* Scrape the addresses of Amazon Headquarter buildings from [link](https://foursquare.com/npietran/list/amazon-buildings) and use geocoder to convert addresses into points with Lat & Long -> amazonPoints.shp
* Perform Spatial Join between amazonPoints.shp and WA blocks.shp to extract blocks that contains amazonPoints -> amazonID.shp
* Refer to ReadCSV.ipynb for processing CSV files
* Create shapefiles that contain Amazon headquarter blocks and Employees' home blocks


* **amazonPoints.shp**: locations of Amazon buildings
* **amazonID.shp**: locations of Amazon Buildings with block ID attribute
* **homeBlocks.shp**: blocks show where Amazon Headquarter employees located
* **amazonBlock.shp**: blocks show where Amazon Headquarter buidling located

In [3]:
import urllib2, lxml, json, shapely, shapely.geometry, os, requests, pandas, geopandas, geocoder
from lxml import html
from pandas import DataFrame
from shapely.geometry import Point
from geopandas import GeoDataFrame

In [4]:
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\ArcToolbox\\Scripts')
import arcpy

In [5]:
# Set up working environemnt
arcpy.env.workspace = "U:\\GEOG458\\Final"
arcpy.env.overwriteOutput = True

In [6]:
# Scrape amazon headquarter addresses from the internet
r = requests.get("https://foursquare.com/npietran/list/amazon-buildings")
raw = r.text

name_start = "</span>&nbsp;"
name_end = "</a>"
start = 0
addr_start = '<span class="address">'
addr_end = "</span>"

name_addr = []

while True:
    start = raw.find(name_start, start)
    if start == -1:
        break
    start += len(name_start)
    
    end = raw.find(name_end, start)
    if end == -1:
        break
    
    name = raw[start:end]
    start = end + len(name_end)
    
    start = raw.find(addr_start, start)
    start += len(addr_start)
    end = raw.find(addr_end, start)
    
    addr = raw[start:end]
    start = end + len(addr_end)
    
    name_addr.append([name, addr])

del name_addr[14]

In [20]:
# Clean up addresses and put them into a dictionary
import re
data = {'Name':[], 'Address':[], 'lat':[], 'lng':[]}
i = 0
while i < len(name_addr):
    data['Name'].append(name_addr[i][0])
    data['Address'].append(re.sub("[\(\[].*?[\)\]]", "",name_addr[i][1])) # Remove text inside ()
    g = geocoder.arcgis(re.sub("[\(\[].*?[\)\]]", "",name_addr[i][1]), maxRows=1)
    data['lat'].append(g.lat)
    data['lng'].append(g.lng)
    i += 1

In [21]:
# Convert dictionary into geoDataFrame
gdf = pandas.DataFrame(data,columns=['Name','Address','lat','lng'])
geometry = [Point(xy) for xy in zip(gdf['lng'], gdf['lat'])] # Create point geometry
crs = {'init': 'epsg:4269'}                                  # Set projection
gdf = GeoDataFrame(gdf, crs=crs, geometry=geometry)

In [22]:
# Convert geoDataFrame to point shp
gdf.to_file('amazonPoints.shp', driver = 'ESRI Shapefile')

In [31]:
# Add block GEOID to point shp
arcpy.SpatialJoin_analysis('amazonPoints.shp', 'block.shp', 'amazonID.shp', '#', '#', '#', 'WITHIN')

<Result 'U:\\GEOG458\\Final\\amazonID.shp'>

###### We used 64bit environment to work with the csv files. New CSV file names:
* OD15_Short.csv: Clean version of the original csv file from LODES
* TotalJobsHomeBlocks.csv: Contains number of workers that live in the same block
* TotalJobsWorkBlocks.csv: Contains number of workers that work in the same block
###### Refer to ReadCSV.ipynb for the codes

In [19]:
# Open csv file of all home blocks with total number of jobs
homeFile = pandas.read_csv("U:\\GEOG458\\Final\\TotalJobsHomeBlocks.csv")
homeId =  homeFile['Home'].apply(str).tolist() # Convert GEOId field from number to string

In [22]:
# Query out and create a new shapefile of blocks for where Amazon employees located
# using GEOId field from the csv file
arcpy.FeatureClassToFeatureClass_conversion('block.shp', 'U:\\GEOG458\\Final\\',
                                            'homeBlocks.shp',
                                            '"GEOID10" IN ' + str(tuple(homeId)))

<Result 'U:\\GEOG458\\Final\\homeBlocks.shp'>

In [23]:
# Open csv file of all Amazon headquarter blocks and total number of jobs
workFile = pandas.read_csv("U:\\GEOG458\\Final\\TotalJobsWorkBlocks.csv")
workId =  workFile['Work'].apply(str).tolist()

In [24]:
# Query out and create a new shapefile of Amazon headquarter blocks
# using GEOId field from the csv file
arcpy.FeatureClassToFeatureClass_conversion('block.shp',
                                            'U:\\GEOG458\\Final\\',
                                            'amazonBlock.shp',
                                            '"GEOID10" IN ' + str(tuple(workId)))

<Result 'U:\\GEOG458\\Final\\workBlocks.shp'>